In [67]:
# installing jdk
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#downloading .tgz installation file for Spache spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz
# installing apache spark from downloaded file
!tar xf spark-3.5.4-bin-hadoop3.tgz
# installing findspark library
!pip install -q findspark

In [68]:
# importin necessary libraries fro this notebook
import os
import findspark
import numpy as np
import pandas as pd
from pyspark.sql.types import StringType # Import StringType from pyspark.sql.types
from pyspark.sql.window import Window # Import the Window class
from pyspark.sql.functions import when, first, col # Import necessary functions



In [69]:
#setting up paths for JDK and spark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"
#initiating findspark
#findspark.init()
findspark.init()

In [70]:
from pyspark.sql import SparkSession
# Create a Spark session
spark = SparkSession.builder.appName("SparkDayOne").getOrCreate()

In [71]:
spark.getActiveSession

<bound method SparkSession.getActiveSession of <class 'pyspark.sql.session.SparkSession'>>

# Import train data

In [72]:
df_train = spark.read.csv("train.csv", header=True, inferSchema=True)
df_train.show()

+------+-----------+--------+---------------+----+-----------+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+------------------+------------+
|    ID|Customer_ID|   Month|           Name| Age|        SSN|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|   Monthly_Balance|Credit_Score|
+------+-----------+--------+---------------+----+-----------+

In [93]:
# Cleaning dle SSN

prepsat_ssn_value = "#F%$D@*&8"
colums_doplnit = ["Customer_ID", "Name", "Occupation", "SSN", "Age"]
window_part = Window.partitionBy("Customer_ID")

df_filled = df_train.withColumn(
    "Name", when(col("Name").isNull(), first("Name", True).over(window_part)).otherwise(col("Name"))
).withColumn(
    "Occupation", when(col("Occupation").isNull(), first("Occupation", True).over(window_part)).otherwise(col("Occupation"))
).withColumn(
    "SSN",
    when((col("SSN").isNull()) | (col("SSN") == prepsat_ssn_value), first(when(col("SSN") != prepsat_ssn_value, col("SSN")), True).over(window_part)).otherwise(col("SSN"))
).withColumn(
    "Age", when(col("Age").isNull(), first("Age", True).over(window_part)).otherwise(col("Age"))
)

df_filled.show()

+-------+-----------+--------+--------------+---+-----------+-------------+------------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+------------------+------------+
|     ID|Customer_ID|   Month|          Name|Age|        SSN|   Occupation|     Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|   Monthly_Balance|Credit_Score|
+-------+-----------+--------+--------------+---

In [74]:
# prompt: Show combinations of Customer_ID and SSN where single SSN is mapped to multiple different Customer_ID values

from pyspark.sql.functions import count

# Group by SSN and count the distinct Customer_IDs
ssn_counts = df_filled.groupBy("SSN").agg(count("Customer_ID").alias("customer_count"))

# Filter for SSNs with more than one distinct Customer_ID
multiple_customer_ids = ssn_counts.filter("customer_count > 1")

# Join with the original DataFrame to get the Customer_ID and SSN combinations
result_df = multiple_customer_ids.join(df_filled, "SSN", "inner").select("Customer_ID", "SSN")

# Show the results
result_df.show()

+-----------+-----------+
|Customer_ID|        SSN|
+-----------+-----------+
| CUS_0x1000|913-74-1218|
| CUS_0x1000|913-74-1218|
| CUS_0x1000|913-74-1218|
| CUS_0x1000|913-74-1218|
| CUS_0x1000|913-74-1218|
| CUS_0x1000|913-74-1218|
| CUS_0x1000|913-74-1218|
| CUS_0x1000|913-74-1218|
| CUS_0x1009|063-67-6938|
| CUS_0x1009|063-67-6938|
| CUS_0x1009|063-67-6938|
| CUS_0x1009|063-67-6938|
| CUS_0x1009|063-67-6938|
| CUS_0x1009|063-67-6938|
| CUS_0x1009|063-67-6938|
| CUS_0x1009|063-67-6938|
| CUS_0x100b|238-62-0395|
| CUS_0x100b|238-62-0395|
| CUS_0x100b|238-62-0395|
| CUS_0x100b|238-62-0395|
+-----------+-----------+
only showing top 20 rows



In [75]:
# prompt: Find unique combinations of Customer_ID and SSN where a single SSN is mapped to multiple Customer_ID values

from pyspark.sql.functions import count, col

# Group by SSN and count the number of unique Customer_IDs
ssn_counts = df_filled.groupBy("SSN").agg(count("Customer_ID").alias("customer_count"))

# Filter for SSNs with more than one Customer_ID
multiple_customer_ssns = ssn_counts.filter(col("customer_count") > 1)

# Join with the original DataFrame to get the Customer_IDs associated with these SSNs
result_df = multiple_customer_ssns.join(df_train, "SSN", "inner").select("Customer_ID", "SSN")

# Show the results
result_df.orderBy(col("SSN")).show()

+-----------+-----------+
|Customer_ID|        SSN|
+-----------+-----------+
| CUS_0x1e84|000-08-1349|
| CUS_0x1e84|000-08-1349|
| CUS_0x1e84|000-08-1349|
| CUS_0x1e84|000-08-1349|
| CUS_0x1e84|000-08-1349|
| CUS_0x1e84|000-08-1349|
| CUS_0x1e84|000-08-1349|
| CUS_0x1e84|000-08-1349|
| CUS_0x11a4|000-40-3312|
| CUS_0x11a4|000-40-3312|
| CUS_0x11a4|000-40-3312|
| CUS_0x11a4|000-40-3312|
| CUS_0x11a4|000-40-3312|
| CUS_0x11a4|000-40-3312|
| CUS_0x11a4|000-40-3312|
| CUS_0x11a4|000-40-3312|
| CUS_0x1383|000-58-0568|
| CUS_0x1383|000-58-0568|
| CUS_0x1383|000-58-0568|
| CUS_0x1383|000-58-0568|
+-----------+-----------+
only showing top 20 rows



# Clean up underscores

In [76]:
def find_underscore_columns(spark_df):

    underscore_columns = []
    for col in spark_df.columns:
        # Check if column is of string type
        if isinstance(spark_df.schema[col].dataType, (StringType)): # Use the imported StringType
          # Convert the column to pandas series to enable string functions
          pandas_series = spark_df.select(col).toPandas()[col]
          if any('_' in str(x) for x in pandas_series):
            underscore_columns.append(col)

    return underscore_columns

# Example usage (assuming df_train is your DataFrame)
underscore_cols = find_underscore_columns(df_filled)

# Remove 'Customer_ID' if present
if 'Customer_ID' in underscore_cols:
    underscore_cols.remove('Customer_ID')

underscore_cols

['Age',
 'Occupation',
 'Annual_Income',
 'Num_of_Loan',
 'Num_of_Delayed_Payment',
 'Changed_Credit_Limit',
 'Credit_Mix',
 'Outstanding_Debt',
 'Amount_invested_monthly',
 'Payment_Behaviour',
 'Monthly_Balance']

In [77]:
from pyspark.sql.functions import regexp_replace

def remove_underscores_from_columns(spark_df, cols_to_modify):
    new_df = spark_df
    for col in cols_to_modify:
        new_df = new_df.withColumn(col, regexp_replace(col, "_", ""))
    return new_df

new_df_train = remove_underscores_from_columns(df_filled, underscore_cols)
new_df_train.show()

+-------+-----------+--------+--------------+---+-----------+------------+------------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+------------------+------------+
|     ID|Customer_ID|   Month|          Name|Age|        SSN|  Occupation|     Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|   Monthly_Balance|Credit_Score|
+-------+-----------+--------+--------------+---+-

Clean from underscores

# Statistics for individual columns

In [78]:
# Identifikace numerických sloupců
numerical_cols = [field.name for field in new_df_train.schema.fields if "IntegerType" in str(field.dataType) or "DoubleType" in str(field.dataType)]

# Popis numerických sloupců
for col_name in numerical_cols:
    print(f"Statistiky pro sloupec: {col_name}")
    new_df_train.select(col_name).describe().show()

Statistiky pro sloupec: Monthly_Inhand_Salary
+-------+---------------------+
|summary|Monthly_Inhand_Salary|
+-------+---------------------+
|  count|                84998|
|   mean|    4194.170849600515|
| stddev|    3183.686166873265|
|    min|    303.6454166666666|
|    max|   15204.633333333333|
+-------+---------------------+

Statistiky pro sloupec: Num_Bank_Accounts
+-------+-----------------+
|summary|Num_Bank_Accounts|
+-------+-----------------+
|  count|           100000|
|   mean|         17.09128|
| stddev|117.4048344273482|
|    min|               -1|
|    max|             1798|
+-------+-----------------+

Statistiky pro sloupec: Num_Credit_Card
+-------+------------------+
|summary|   Num_Credit_Card|
+-------+------------------+
|  count|            100000|
|   mean|          22.47443|
| stddev|129.05741011009866|
|    min|                 0|
|    max|              1499|
+-------+------------------+

Statistiky pro sloupec: Interest_Rate
+-------+------------------+
|

# Identify data types

In [79]:
print("Data Types:")
new_df_train.printSchema()

Data Types:
root
 |-- ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Annual_Income: string (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: integer (nullable = true)
 |-- Num_Credit_Card: integer (nullable = true)
 |-- Interest_Rate: integer (nullable = true)
 |-- Num_of_Loan: string (nullable = true)
 |-- Type_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: string (nullable = true)
 |-- Changed_Credit_Limit: string (nullable = true)
 |-- Num_Credit_Inquiries: double (nullable = true)
 |-- Credit_Mix: string (nullable = true)
 |-- Outstanding_Debt: string (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Credit_History_Age: string (nul

# Basic statistics for numerical columns

In [80]:
new_df_train.describe().show()

+-------+-------+-----------+------+------+-----------------+-----------+----------+------------------+---------------------+------------------+------------------+------------------+-----------------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+-----------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+--------------------+------------+
|summary|     ID|Customer_ID| Month|  Name|              Age|        SSN|Occupation|     Annual_Income|Monthly_Inhand_Salary| Num_Bank_Accounts|   Num_Credit_Card|     Interest_Rate|      Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix| Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|     Monthly_Balance|Credit_S

# Find duplicates

In [81]:
df_duplicates = new_df_train.groupBy(df_train.columns).count().filter("count > 1")
df_duplicates.show()

+---+-----------+-----+----+---+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+------------------+---------------------+-------------------+-----------------------+-----------------+---------------+------------+-----+
| ID|Customer_ID|Month|Name|Age|SSN|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|Payment_Behaviour|Monthly_Balance|Credit_Score|count|
+---+-----------+-----+----+---+---+----------+-------------+---------------------+-----------------+---------------+-------------+---

# Find missing values

In [82]:
from pyspark.sql.functions import col, sum

missing_values = new_df_train.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_train.columns])
missing_values.show()

+---+-----------+-----+----+---+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+------------------+---------------------+-------------------+-----------------------+-----------------+---------------+------------+
| ID|Customer_ID|Month|Name|Age|SSN|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|Payment_Behaviour|Monthly_Balance|Credit_Score|
+---+-----------+-----+----+---+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+---

In [91]:
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType

# Seznam sloupců, které chceme převést na číselné hodnoty
numeric_columns = ["Age", "Annual_Income", "Monthly_Inhand_Salary",
                   "Num_Bank_Accounts", "Num_Credit_Card", "Interest_Rate",
                   "Num_of_Loan", "Num_of_Delayed_Payment",
                   "Changed_Credit_Limit", "Num_Credit_Inquiries",
                   "Outstanding_Debt", "Credit_Utilization_Ratio",
                   "Credit_History_Age", "Total_EMI_per_month",
                   "Amount_invested_monthly", "Monthly_Balance"]

# Převod každého sloupce na číselný typ s ošetřením chybějících nebo neplatných hodnot
for column in numeric_columns:
    data = data.withColumn(column, col(column).cast("double"))

data.show()

+-------+-----------+--------+--------------+----+-----------+------------+------------------+---------------------+-----------------+---------------+-------------+-----------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+------------------+---------------------+-------------------+-----------------------+--------------------+------------------+------------+
|     ID|Customer_ID|   Month|          Name| Age|        SSN|  Occupation|     Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|   Monthly_Balance|Credit_Score|
+-------+-----------+--------+--------------+----+--

# Data types after type casting

In [92]:
data.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SSN: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Annual_Income: double (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: double (nullable = true)
 |-- Num_Credit_Card: double (nullable = true)
 |-- Interest_Rate: double (nullable = true)
 |-- Num_of_Loan: double (nullable = true)
 |-- Type_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: double (nullable = true)
 |-- Changed_Credit_Limit: double (nullable = true)
 |-- Num_Credit_Inquiries: double (nullable = true)
 |-- Credit_Mix: string (nullable = true)
 |-- Outstanding_Debt: double (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Credit_History_Age: double (nullable = true)
 